In [4]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.co

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [7]:
from pyspark.sql.functions import to_date
import pandas as pd
# Read in the Review dataset as a DataFrame
df.dropna
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [8]:
# Recreate the vine_table, and perform your analysis
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [9]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
# print((vine_df.count(), len(vine_df.columns)))
total_reviews = vine_df.count()
print(total_reviews)


1785997


In [27]:
# Load in sql function to use columns
from pyspark.sql.functions import col

# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or 
# greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
helpful_reviews_df = vine_df.filter(col("total_votes") >= 20)
print(helpful_reviews_df.count())

65379


In [33]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where 
# a review was written as part of the Vine program (paid), vine == 'Y'.
vine_program_df = helpful_reviews_df.filter(col("vine") == 'Y')
vine_program_df.show()

total_vine_count = vine_program_df.count()
print(total_vine_count)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
|R371M3ZU3ULQHM|          1|            0|         25|   Y|                N|
|R18YQ4NRL2373C|          3|            2|         28|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|     

In [46]:
# 5-star Vine review DataFrame
five_star_vine_reviews = vine_program_df.filter(col("star_rating") == 5)
five_star_vine_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|                N|
|R1OWK33OPI45KT|          5|           55|         58|   Y|                N|
|R2UP7VTED8O425|          5|           35|         41|   Y|                N|
|R3N4QAXTQCA1ED|          5|           18|         20|   Y|                N|
|R3EPHK0JGQGE91|          5|           21|         28|   Y|     

In [44]:
# 5-star paid review count
total_five_star_vine_reviews = (five_star_vine_reviews.count())
print(total_five_star_vine_reviews)

48


In [45]:
# Percentage of paid 5-star reviews 
five_star_vine_review_percent = (total_five_star_vine_reviews / total_vine_count) * 100
print(five_star_vine_review_percent)

46.15384615384615


In [51]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
not_vine_program_df = helpful_reviews_df.filter(col("vine") == 'N')
not_vine_program_df.show()

not_vine_count = not_vine_program_df.count()
print(not_vine_count)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [48]:
# 5-star unpaid reviews DataFrame
five_star_reviews = not_vine_program_df.filter(col("star_rating") == 5)
five_star_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R29BOS5HMAY1LO|          5|           88|        110|   N|                N|
| RDX2ZZ46AM343|          5|           32|         34|   N|                Y|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|
|R2FAARI3JQO9XQ|          5|           29|         30|   N|                Y|
|R2WS53BH47WUMT|          5|           68|         84|   N|                N|
|R2BBH73BUJYBGK|          5|           18|         22|   N|                N|
|R18ZBSSAMJOV8I|          5|          450|        471|   N|                N|
|R3M42FPE0VJA0U|          5|           20|         22|   N|                Y|
| RCK4WXLGD3HDY|          5|           72|         81|   N|     

In [49]:
# 5-star unpaid review count
total_five_star_reviews = (five_star_reviews.count())
print(total_five_star_reviews)

20439


In [53]:
# Percentage of unpaid 5-star reviews 
five_star_review_percent = (total_five_star_reviews / not_vine_count) * 100
print(five_star_review_percent)

31.312140942167755
